# BiLSTM-CRF 
[ADVANCED: MAKING DYNAMIC DECISIONS AND THE BI-LSTM CRF](https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html)  
[Implementing a linear-chain Conditional Random Field (CRF) in PyTorch](https://towardsdatascience.com/implementing-a-linear-chain-conditional-random-field-crf-in-pytorch-16b0b9c4b4ea)


## Linear-Chain Conditional Random Field (CRF) 

The source sequence is $x = \{x_1, x_2, \dots, x_T \}$, and the target sequence is $y = \{y_1, y_2, \dots, y_T \}$.  
If we ignore the dependence between elements in $y$, we can model as:
$$
\begin{aligned}
P(y|x) &= \prod_{t=1}^T \frac{\exp \left( U(x_t, y_t) \right)}{\sum_{y'_t} \exp \left( U(x_t, y'_t) \right)} \\
&= \prod_{t=1}^T \frac{\exp \left( U(x_t, y_t) \right)}{Z(x_t)} \\
&= \frac{\exp \left( \sum_{t=1}^T U(x_t, y_t) \right)}{\prod_{t=1}^T Z(x_t)} \\
&= \frac{\exp \left( \sum_{t=1}^T U(x_t, y_t) \right)}{Z(x)}
\end{aligned}
$$
where $U(x_t, y_t)$ is *emissions* or *unary scores*, $Z(x_t)$ is *partition function* (a normalization factor).  
In a *linear-chain CRF*, we add *transition scores* $T(y_t, y_{t+1})$ to the above equation:  
$$
P(y|x) = \frac{\exp \left( \sum_{t=1}^T U(x_t, y_t) + \sum_{t=0}^T T(y_t, y_{t+1}) \right)}{Z(x)}
$$
where $y_0$ and $y_{T+1}$ are the starting and stopping tags, respectively; their values are fixed.  
The partition function should sum over all possible combinations over the label set at each timestep: 
$$
Z(x) = \sum_{y'_1} \sum_{y'_2} \dots \sum_{y'_T} \exp \left( \sum_{t=1}^T U(x_t, y'_t) + \sum_{t=0}^T T(y'_t, y'_{t+1}) \right)
$$

The *negative log-likelihood loss (NLL-Loss)* is: 
$$
\begin{aligned}
L &= -\log \left( P(y|x) \right) \\
&= \log \left( Z(x) \right) - \left( \sum_{t=1}^T U(x_t, y_t) + \sum_{t=0}^T T(y_t, y_{t+1}) \right)
\end{aligned}
$$

## Forward Algorithm: Dynamic Programing for Computing the Partition Function

The time complexity of computing $Z(x)$ would be $O(\vert y \vert^T)$... but we can use *dynamic programing* to reduce it.  
Specifically, we define the state:
$$
\alpha_s (y_s) = \sum_{y'_1} \sum_{y'_2} \dots \sum_{y'_{s-1}} \exp \left( \sum_{t=1}^{s-1} U(x_t, y'_t) + \sum_{t=0}^{s-2} T(y'_t, y'_{t+1}) + T(y'_{s-1}, y_s) \right)
$$
where $\alpha_s (y_s)$ may be regarded as the sum of scores reaching $y_s$. Note:  
$$
\begin{aligned}
\alpha_1(y_1) &= \exp \left( T(y_0, y_1) \right) \\
\alpha_{T+1}(y_{T+1}) &= Z(x)
\end{aligned}
$$
When computing $\alpha_{s+1}(y_{s+1})$, we only require the information of $\alpha_s(y'_s)$ for different $y'_s$, instead of the information before step $s$ (i.e., the paths reaching each $y'_s$). Hence, this is a dynamic programing problem. 
In the log-space, we have the *state transition equation*:  
$$
\begin{aligned}
\log( \alpha_{s+1}(y_{s+1}) ) &= \log \left( \sum_{y'_1} \sum_{y'_2} \dots \sum_{y'_s} \exp \left( \sum_{t=1}^s U(x_t, y'_t) + \sum_{t=0}^{s-1} T(y'_t, y'_{t+1}) + T(y'_s, y_{s+1}) \right) \right) \\
&= \log \left( \sum_{y'_s} \exp \left( U(x_s, y'_s) + T(y'_s, y_{s+1}) \right) \sum_{y'_1} \sum_{y'_2} \dots \sum_{y'_{s-1}} \exp \left( \sum_{t=1}^{s-1} U(x_t, y'_t) + \sum_{t=0}^{s-2} T(y'_t, y'_{t+1}) + T(y'_{s-1}, y'_s) \right) \right) \\
&= \log \left( \sum_{y'_s} \exp \left( U(x_s, y'_s) + T(y'_s, y_{s+1}) \right) \alpha_s(y'_s) \right) \\
&= \log \left( \sum_{y'_s} \exp \left( U(x_s, y'_s) + T(y'_s, y_{s+1}) + \log (\alpha_s(y'_s)) \right) \right)
\end{aligned}
$$

## Viterbi Algorithm: Finding the Best Sequence of Labels 

Similarly, we use dynamic programing to find the best sequence of labels (i.e., decoding).  
Specifically, we define the state: 
$$
\beta_s (y_s) = \max_{y'_1, y'_2, \dots, y'_{s-1}} \exp \left( \sum_{t=1}^{s-1} U(x_t, y'_t) + \sum_{t=0}^{s-2} T(y'_t, y'_{t+1}) + T(y'_{s-1}, y_s) \right)
$$
where $\beta_s (y_s)$ may be regarded as the max score reaching $y_s$. Note: 
$$
\begin{aligned}
\beta_1 (y_1) &= \exp \left( T(y_0, y_1) \right) \\
\beta_{T+1}(y_{T+1}) &= \max_{y'_1, y'_2, \dots, y'_T} \exp \left( \sum_{t=1}^T U(x_t, y'_t) + \sum_{t=0}^{T-1} T(y'_t, y'_{t+1}) + T(y'_T, y_{T+1}) \right)
\end{aligned} 
$$

In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

SEED = 515
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Preparing Data

In [2]:
training_data = [("the wall street journal reported today that apple corporation made money".split(),
                  "B I I I O O O B I O O".split()), 
                 ("georgia tech is a university in georgia".split(),
                  "B I O O O O B".split())]
training_data

[(['the',
   'wall',
   'street',
   'journal',
   'reported',
   'today',
   'that',
   'apple',
   'corporation',
   'made',
   'money'],
  ['B', 'I', 'I', 'I', 'O', 'O', 'O', 'B', 'I', 'O', 'O']),
 (['georgia', 'tech', 'is', 'a', 'university', 'in', 'georgia'],
  ['B', 'I', 'O', 'O', 'O', 'O', 'B'])]

In [3]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"

word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}

## Building the Model

In [4]:
EMB_DIM = 5
HID_DIM = 4
VOC_DIM = len(word_to_ix)
TAG_DIM = len(tag_to_ix)

emb = nn.Embedding(VOC_DIM, EMB_DIM)
rnn = nn.LSTM(EMB_DIM, HID_DIM//2, num_layers=1, bidirectional=True)
hid2tag = nn.Linear(HID_DIM, TAG_DIM)

In [5]:
ex_idx = 0
# sent/tags: (step, batch=1)
sent = torch.tensor([word_to_ix[w] for w in training_data[ex_idx][0]], dtype=torch.long).unsqueeze(1)
tags = torch.tensor([tag_to_ix[t] for t in training_data[ex_idx][1]], dtype=torch.long).unsqueeze(1)
print(sent)
print(tags)

tensor([[ 0],
        [ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5],
        [ 6],
        [ 7],
        [ 8],
        [ 9],
        [10]])
tensor([[0],
        [1],
        [1],
        [1],
        [2],
        [2],
        [2],
        [0],
        [1],
        [2],
        [2]])


In [6]:
embbed = emb(sent)
rnn_outs, _ = rnn(embbed)

# feats: (step, batch=1, tag_dim)
feats = hid2tag(rnn_outs)
print(feats)

tensor([[[ 0.1153,  0.1015, -0.2363, -0.1656, -0.3136]],

        [[-0.1415,  0.1355, -0.2845, -0.2094, -0.2525]],

        [[-0.0358,  0.0669, -0.1656, -0.1879, -0.2165]],

        [[ 0.0696, -0.0096,  0.0631,  0.0470, -0.1283]],

        [[-0.0778,  0.0396, -0.0088,  0.0143, -0.1432]],

        [[ 0.0399,  0.1006, -0.1594, -0.1141, -0.3252]],

        [[-0.1022,  0.0704, -0.1275, -0.0914, -0.1637]],

        [[-0.0796,  0.0148, -0.0523, -0.1221, -0.1086]],

        [[-0.1715,  0.0859, -0.0614,  0.0529, -0.1484]],

        [[-0.2149,  0.1116, -0.0498,  0.1023, -0.1867]],

        [[-0.0229, -0.0485,  0.1840,  0.1056, -0.0520]]],
       grad_fn=<AddBackward0>)


In [7]:
# transitions[i, j] is the score of transitioning *to* i *from* j.
transitions = nn.Parameter(torch.randn(TAG_DIM, TAG_DIM))
transitions.data[tag_to_ix[START_TAG], :] = -1e4
transitions.data[:, tag_to_ix[STOP_TAG]] = -1e4
transitions

Parameter containing:
tensor([[-1.1290e+00, -1.2998e+00, -1.6684e+00, -8.6616e-01, -1.0000e+04],
        [-4.6060e-01,  4.6536e-01, -1.5174e+00, -1.5891e+00, -1.0000e+04],
        [-1.2220e+00, -2.4080e-01, -8.2866e-01,  7.9692e-01, -1.0000e+04],
        [-1.0000e+04, -1.0000e+04, -1.0000e+04, -1.0000e+04, -1.0000e+04],
        [-5.8353e-01, -6.6203e-01,  1.1687e-01,  1.1368e+00, -1.0000e+04]],
       requires_grad=True)

In [8]:
# The numerator: score
# The original implementation
def _score_sentence(feats, tags):
    score = torch.zeros(1)
    tags = torch.cat([torch.tensor([tag_to_ix[START_TAG]], dtype=torch.long), tags])
    for i, feat in enumerate(feats):
        score = score + transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
    score = score +transitions[tag_to_ix[STOP_TAG], tags[-1]]
    return score

_score_sentence(feats.squeeze(1), tags.squeeze(1))

tensor([-5.2230], grad_fn=<AddBackward0>)

In [9]:
# Vectorized implementation
feat_scores = feats.gather(dim=-1, index=tags.unsqueeze(-1)).squeeze(-1)
print(feat_scores.size())

from_tags = torch.cat([torch.full((1, tags.size(1)), fill_value=tag_to_ix[START_TAG], dtype=torch.long), tags], dim=0)
to_tags = torch.cat([tags, torch.full((1, tags.size(1)), fill_value=tag_to_ix[STOP_TAG], dtype=torch.long)], dim=0)
trans_scores = transitions[to_tags, from_tags]
print(trans_scores.size())

print(feat_scores.sum(dim=0) + trans_scores.sum(dim=0))

torch.Size([11, 1])
torch.Size([12, 1])
tensor([-5.2230], grad_fn=<AddBackward0>)


In [10]:
# The denominator: partition function
# The original implementation
def log_sum_exp(vec):
    max_score = vec[0, torch.argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

def _forward_alg(feats):
    # Do the forward algorithm to compute the partition function
    init_alphas = torch.full((1, TAG_DIM), -10000.)
    # START_TAG has all of the score.
    init_alphas[0][tag_to_ix[START_TAG]] = 0.

    # Wrap in a variable so that we will get automatic backprop
    forward_var = init_alphas

    # Iterate through the sentence
    for feat in feats:
        alphas_t = []  # The forward tensors at this timestep
        for next_tag in range(TAG_DIM):
            # broadcast the emission score: it is the same regardless of
            # the previous tag
            emit_score = feat[next_tag].view(1, -1).expand(1, TAG_DIM)
            # the ith entry of trans_score is the score of transitioning to
            # next_tag from i
            trans_score = transitions[next_tag].view(1, -1)
            # The ith entry of next_tag_var is the value for the
            # edge (i -> next_tag) before we do log-sum-exp
            next_tag_var = forward_var + trans_score + emit_score
            # The forward variable for this tag is log-sum-exp of all the
            # scores.
            alphas_t.append(log_sum_exp(next_tag_var).view(1))
        forward_var = torch.cat(alphas_t).view(1, -1)
    terminal_var = forward_var + transitions[tag_to_ix[STOP_TAG]]
    alpha = log_sum_exp(terminal_var)
    return alpha

_forward_alg(feats.squeeze(1))

tensor(6.5684, grad_fn=<AddBackward0>)

In [11]:
# Vectorized implementation
alphas = torch.full((feats.size(1), TAG_DIM), fill_value=-1e4)
alphas[:, tag_to_ix[START_TAG]] = 0
print(alphas)

for t in range(feats.size(0)):
    # alphas: (batch=1, tag_dim)
    # feats[t]: (batch=1, tag_dim)
    alphas = torch.logsumexp(alphas.unsqueeze(1) + feats[t].unsqueeze(2) + transitions, dim=-1)

alphas = alphas + transitions[tag_to_ix[STOP_TAG]]
torch.logsumexp(alphas, dim=-1)

tensor([[-10000., -10000., -10000.,      0., -10000.]])


tensor([6.5684], grad_fn=<LogsumexpBackward>)